In [ ]:
!pip install git+https://github.com/suno-ai/bark.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-zr6jn5sk
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-zr6jn5sk
  Resolved https://github.com/suno-ai/bark.git to commit f6f2db527b13c4a3e52ed6fbac587aadc3723eb6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 74.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 109.0 MB/s

In [ ]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from bark.api import semantic_to_waveform
from bark.generation import (
    generate_text_semantic,
    preload_models,
)

from IPython.display import Audio
import nltk 
import numpy as np

preload_models()

Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:00<00:00, 155MB/s]


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Run on GPU else too slow

In [ ]:
text_prompt = """
     Hello [laughs] world! I'm feeling ... GOOD!
"""
audio_array = generate_audio(text_prompt)
Audio(audio_array, rate=SAMPLE_RATE)

100%|██████████| 18/18 [00:14<00:00,  1.21it/s]


In [ ]:
text_prompt = """
    WOMAN: I would like an oatmilk latte please.
    MAN: Wow, that's expensive!
"""
audio_array = generate_audio(text_prompt)
Audio(audio_array, rate=SAMPLE_RATE)

In [ ]:
text_prompt = """
    I have a silky smooth voice, and today I will tell you 
    about the exercise regimen of the common sloth.
"""
audio_array = generate_audio(text_prompt, history_prompt="v2/en_speaker_1")
Audio(audio_array, rate=SAMPLE_RATE)

In [ ]:
text_prompt = """
     Scusa, mi potresti passare il sale?
"""
audio_array = generate_audio(text_prompt)
Audio(audio_array, rate=SAMPLE_RATE)

# Longer Audios

In [ ]:
SPEAKER = "v2/en_speaker_6"
GEN_TEMP = 0.6

In [ ]:
def tokenize(script):

  return nltk.sent_tokenize(script)

In [ ]:
def create_tokens(sentences: list):

  silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence

  pieces = []
  for sentence in sentences:
    semantic_tokens = generate_text_semantic(
      sentence,
      history_prompt=SPEAKER,
      temp=GEN_TEMP,
      min_eos_p=0.05,  # this controls how likely the generation is to end
      )

    audio_array = semantic_to_waveform(semantic_tokens, history_prompt=SPEAKER,)
    pieces += [audio_array, silence.copy()]

  return pieces

In [ ]:
def concat_samples(samples):

  return Audio(np.concatenate(samples), rate=SAMPLE_RATE)

In [ ]:
script = """
Hello, my name is Tom and I am here to talk to you about the wonderful country called
Japan, the land of samurai. Samurai were the hereditary military nobility and officer caste of medieval and early-modern 
Japan from the late 12th century until their abolition in the 1870s during the Meiji era. 
They were the well-paid retainers of the daimyo, the great feudal landholders. They had high prestige and special privileges.
""".replace("\n", " ").strip()

In [ ]:
sentences = tokenize(script)

In [ ]:
samples = create_tokens(sentences)

100%|██████████| 9/9 [00:08<00:00,  1.05it/s]


In [ ]:
concat_samples(samples)